# Debugging

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision

DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

**Q: "No debugger for your code. What do you think?"**

**A: "I would NOT be able to code!"**

- Who does "print-line-debugging"?
- Who likes debugging in tensorflow?
- What is the intersection of those two groups?


## IPDB cheatsheet
IPython Debugger

Taken from http://wangchuan.github.io/coding/2017/07/12/ipdb-cheat-sheet.html

- h(help): Print help

- n(ext): Continue execution until the next line in the current function is reached or it returns.
- s(tep): Execute the current line, stop at the first possible occasion (either in a function that is called or in the current function).
- r(eturn): Continue execution until the current function returns.
- c(ont(inue)): Continue execution, only stop when a breakpoint is encountered.

- r(eturn): Continue execution until the current function returns.
- a(rgs): Print the argument list of the current function.

Note: Python 3.7 has `breakpoint()` built-in! [[PEP 553]](https://www.python.org/dev/peps/pep-0553/)

In [2]:
from IPython.core.debugger import set_trace

In [5]:
def my_function(x):
    answer = 42
    set_trace()
    answer += x
    return answer

my_function(12)

> <ipython-input-5-0515935c5419>(4)my_function()
      2     answer = 42
      3     set_trace()
----> 4     answer += x
      5     return answer
      6 



ipdb>  
ipdb>  c


54

## Example: debuging a NN

In [6]:
X = torch.rand((5, 3))
X

tensor([[0.2962, 0.6341, 0.4601],
        [0.3189, 0.2215, 0.0641],
        [0.4122, 0.7333, 0.9693],
        [0.2791, 0.3007, 0.6426],
        [0.8482, 0.4750, 0.9523]])

In [9]:
class MyModule(nn.Module):
    def __init__(self):
        super().__init__()
        self.lin = nn.Linear(3, 1)
    
    def forward(self, X):
        # set_trace()
        X = self.lin(X)
        return X

    
model = MyModule()
y_ = model(X)

assert y_.shape == (5, 1), y_.shape

## Debug Layer

In [10]:
class DebugModule(nn.Module):
    def forward(self, x):
        set_trace()
        return x

In [11]:
model = nn.Sequential(
    nn.Linear(1, 5),
    DebugModule(),
    nn.Linear(5, 1),
)

In [12]:
X = torch.unsqueeze(torch.tensor([1.]), dim=0)
model(X)

> <ipython-input-10-a95e71519cda>(4)forward()
      1 class DebugModule(nn.Module):
      2     def forward(self, x):
      3         set_trace()
----> 4         return x



ipdb>  n


--Return--
tensor([[ 0.1...ddmmBackward>)
> <ipython-input-10-a95e71519cda>(4)forward()
      1 class DebugModule(nn.Module):
      2     def forward(self, x):
      3         set_trace()
----> 4         return x



ipdb>  n


> c:\users\mikhail.galkin\appdata\local\continuum\anaconda3\envs\py-pytorch\lib\site-packages\torch\nn\modules\module.py(490)__call__()
    488         else:
    489             result = self.forward(*input, **kwargs)
--> 490         for hook in self._forward_hooks.values():
    491             hook_result = hook(self, input, result)
    492             if hook_result is not None:



...

--Return--
tensor([[ 0.1...ddmmBackward>)
> c:\users\mikhail.galkin\appdata\local\continuum\anaconda3\envs\py-pytorch\lib\site-packages\torch\nn\modules\module.py(509)__call__()
    507                     functools.update_wrapper(wrapper, hook)
    508                     grad_fn.register_hook(wrapper)
--> 509         return result
    510 
    511     def __setstate__(self, state):



ipdb>  n


> c:\users\mikhail.galkin\appdata\local\continuum\anaconda3\envs\py-pytorch\lib\site-packages\torch\nn\modules\container.py(91)forward()
     89 
     90     def forward(self, input):
---> 91         for module in self._modules.values():
     92             input = module(input)
     93         return input



ipdb>  c


tensor([[0.4005]], grad_fn=<AddmmBackward>)

## Tensorboard and `tensorboardX`
Tensorboard and `tensorboardX` are also great to debug a model, e.g. to look at the gradients.